In [6]:
from transformers import AutoTokenizer
from utils.dataset import NERDataSet

In [2]:
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base', use_fast=True)

In [3]:
text = """Đội ngũ của chúng tôi tại DVSC luôn cố gắng cung cấp cho khách hàng và đối tác những phân tích chính xác nhất đồng thời tạo ra lợi nhuận lâu dài và bền vững .
"""



In [7]:
ner_dataset = NERDataSet(jsonl_file='dataset/all_data_v1_02t03.jsonl')
dataset_df = ner_dataset.dataset_df

In [9]:
conll_format = dataset_df.iloc[0]['conll_label']

In [48]:
i = 5


In [49]:
dataset_df.iloc[i]['conll_label']

['Công B-ORGANIZATION',
 'ty I-ORGANIZATION',
 'cổ I-ORGANIZATION',
 'phần I-ORGANIZATION',
 'truyền I-ORGANIZATION',
 'thông I-ORGANIZATION',
 'đa I-ORGANIZATION',
 'phương I-ORGANIZATION',
 'tiện I-ORGANIZATION',
 'MMS I-ORGANIZATION',
 'đã O',
 'được O',
 'nhiều O',
 'khách B-PERSONTYPE',
 'hàng I-PERSONTYPE',
 'tin O',
 'cậy O',
 'và O',
 'đánh O',
 'giá O',
 'cao O',
 'trong O',
 'suốt O',
 'những O',
 'năm O',
 'phát O',
 'triển O',
 '. O']

In [11]:
word_list = [word.split(' ')[0] for word in conll_format]
label_list = [word.split(' ')[-1] for word in conll_format]

In [12]:
word_list

['Ngày',
 'nay',
 'nhu',
 'cầu',
 'sử',
 'dụng',
 'các',
 'poster',
 ';',
 'background',
 'cho',
 'sự',
 'kiện',
 ',',
 'những',
 'ấn',
 'phẩm',
 'truyền',
 'thông',
 'ngày',
 'càng',
 'được',
 'ứng',
 'dụng',
 'rộng',
 'rãi',
 '.']

In [13]:
tokens = tokenizer(word_list, is_split_into_words=True)

In [16]:
len(tokens['input_ids']), len(label_list)

(31, 27)

In [14]:
tokenizer.convert_ids_to_tokens(tokens['input_ids'])

['<s>',
 '▁Ngày',
 '▁nay',
 '▁nhu',
 '▁cầu',
 '▁sử',
 '▁dụng',
 '▁các',
 '▁poster',
 '▁;',
 '▁background',
 '▁cho',
 '▁sự',
 '▁kiện',
 '▁',
 ',',
 '▁những',
 '▁ấn',
 '▁phẩm',
 '▁truyền',
 '▁thông',
 '▁ngày',
 '▁càng',
 '▁được',
 '▁ứng',
 '▁dụng',
 '▁rộng',
 '▁rãi',
 '▁',
 '.',
 '</s>']

In [17]:
tokens.word_ids()

[None,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 26,
 None]

In [28]:
from typing import List
label_all_tokens = True
def tokenize_and_align_labels(data_point: List[str]):
    word_list = [word.split(' ')[0] for word in data_point]
    label_list = [word.split(' ')[-1] for word in data_point]

    tokenized_inputs = tokenizer(word_list, truncation=True, is_split_into_words=True, max_length=128, padding='max_length')
    word_ids = tokenized_inputs.word_ids()

    previous_word_idx = None
    label_ids = []
    for word_idx in word_ids:
        # Special tokens have a word id that is None. We set the label to -100 so they are automatically
        # ignored in the loss function.
        if word_idx is None:
            label_ids.append(-100)
        # We set the label for the first token of each word.
        elif word_idx != previous_word_idx:
            label_ids.append(label_list[word_idx])
        # For the other tokens in a word, we set the label to either the current label or -100, depending on
        # the label_all_tokens flag.
        else:
            label_ids.append(label_list[word_idx] if label_all_tokens else -100)
        previous_word_idx = word_idx

    tokenized_inputs["labels"] = label_ids
    return tokenized_inputs

In [29]:
tokenized_inputs = tokenize_and_align_labels(data_point=conll_format)

In [30]:
tokenized_inputs

{'input_ids': [0, 42812, 7630, 36382, 7756, 5034, 2786, 925, 40908, 2819, 76615, 681, 2550, 16074, 6, 4, 1358, 71846, 5692, 18099, 4225, 3063, 18432, 912, 13932, 2786, 29564, 128080, 6, 5, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [-100, 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PRODUCT', 'O', 'B-PRODUCT', 'O', 'O', 'O', 'O', 'O', 'O',

In [27]:
len(tokenized_inputs['input_ids'])

31

In [31]:
dataset_df

,id,ref_id,source,text,conll_label
0,0,42.0,tan_data_185_email_BIO,Ngày nay nhu cầu sử dụng các poster ; backgrou...,"[Ngày O, nay O, nhu O, cầu O, sử O, dụng O, cá..."
1,1,42.0,tan_data_185_email_BIO,"Chính vì thế , một loạt các công ty thiết kế đ...","[Chính O, vì O, thế O, , O, một O, loạt O, các..."
2,1,NaN,None,"Môi trường làm việc lý tưởng , văn phòng đẹp ,...","[Môi O, trường O, làm O, việc O, lý O, tưởng O..."
3,2,42.0,tan_data_185_email_BIO,"Và dưới đây , Saigon Review xin giới thiệu top...","[Và O, dưới O, đây O, , O, Saigon B-ORGANIZATI..."
4,3,42.0,tan_data_185_email_BIO,Là công ty thiết kế đồ họa tại TPHCM với thế m...,"[Là O, công O, ty O, thiết O, kế O, đồ O, họa ..."
...,...,...,...,...,...
18784,638758,NaN,None,"Yêu là vượt lên nỗi sợ bị tổn thương , để cá t...","[Yêu O, là O, vượt O, lên O, nỗi O, sợ O, bị O..."
18785,638760,NaN,None,phẩm chất .,"[phẩm O, chất O, . O]"
18786,638762,NaN,None,"""She'd better call me back later!"" (Tốt hơn hế...","[""She'd O, better O, call O, me O, back O, lat..."
18787,638790,NaN,None,e) Giấy chứng nhận sức khỏe do cơ quan y tế có...,"[e) O, Giấy O, chứng O, nhận O, sức O, khỏe O,..."


In [33]:
from sklearn.model_selection import train_test_split

In [38]:
df_train, df_rest = train_test_split(dataset_df, shuffle=True, random_state=43, stratify=dataset_df[['source']], train_size=0.7)

In [44]:
df_train['source'].value_counts(dropna=False)

vnexpress_374_BIO         6509
tan_data_185_email_BIO    3150
NaN                       1394
chotot_filtered_BIO       1138
200_IP_BIO                 546
phapluat_BIO               209
skill_BIO                  172
Tan_Manh_address_BIO        34
Name: source, dtype: int64

In [47]:
df_train

,id,ref_id,source,text,conll_label
16942,17243,207.0,vnexpress_374_BIO,"Trong giai đoạn đầu sự nghiệp , ông làm việc c...","[Trong O, giai O, đoạn O, đầu O, sự O, nghiệp ..."
9003,8831,64.0,vnexpress_374_BIO,"Tuy nhiên , chính quyền mới của Taliban ngay l...","[Tuy O, nhiên O, , O, chính O, quyền O, mới O,..."
17920,347239,NaN,None,"Nếu đó là điều không thể , bạn nên dọn sạch ph...","[Nếu O, đó O, là O, điều O, không O, thể O, , ..."
7899,7649,9.0,tan_data_185_email_BIO,Trường đại học có ngành tài chính ngân hàng TP...,"[Trường O, đại O, học O, có O, ngành O, tài O,..."
18025,347873,NaN,None,"Bão tới , mái tốc , nhà xiêu .","[Bão O, tới O, , O, mái O, tốc O, , O, nhà O, ..."
...,...,...,...,...,...
7364,7104,132.0,vnexpress_374_BIO,"Thạc sĩ Tú Phạm , chủ nhiệm trung tâm , cho bi...","[Thạc B-PERSONTYPE, sĩ I-PERSONTYPE, Tú B-PERS..."
13790,13950,190.0,chotot_filtered_BIO,Công ty mình cần Tuyển Nhân Viên Cung Cấp Suất...,"[Công O, ty O, mình O, cần O, Tuyển O, Nhân B-..."
8152,7937,114.0,vnexpress_374_BIO,Sư thầy Thích Đức Trí cho biết đã được gia đìn...,"[Sư B-PERSONTYPE, thầy I-PERSONTYPE, Thích B-P..."
372,356,52.0,vnexpress_374_BIO,Trường hợp của Wilfred là một trong những vụ b...,"[Trường O, hợp O, của O, Wilfred B-PERSON, là ..."


In [45]:
df_train['source'].describe(dropna=False)

TypeError: describe() got an unexpected keyword argument 'dropna'

In [46]:
df_rest['source'].value_counts(dropna=False)

vnexpress_374_BIO         2790
tan_data_185_email_BIO    1350
NaN                        598
chotot_filtered_BIO        488
200_IP_BIO                 234
phapluat_BIO                89
skill_BIO                   74
Tan_Manh_address_BIO        14
Name: source, dtype: int64

In [1]:
from datasets import load_metric

In [2]:
metric = load_metric("seqeval")

In [3]:
labels = ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PRODUCT', 'O', 'B-PRODUCT', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PRODUCT', 'I-PRODUCT']

In [4]:
metric.compute(predictions=[labels], references=[labels])

{'PRODUCT': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 3},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}